In [3]:
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.types import Command
from typing_extensions import Literal
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

In [ ]:
class CustomState(MessagesState):
    custom_variable_1: str

In [ ]:
def agent_1(state: CustomState) -> Command[Literal['agent_2', '__end__']]:

    messages = [
        SystemMessage('System Prompt'),
        *state['messages']
    ]

    # Invoke model, structure output, etc

    return Command(
        goto='agent_2',
        update={'custom_variable_1': 'agent_1'}
    )

In [ ]:
def agent_2(state: CustomState) -> Command[Literal['__end__']]:

    messages = [
        SystemMessage('System Prompt'),
        *state['messages']
    ]

    # Invoke model, structure output, etc

    return Command(
        goto='__end__',
        update={'custom_variable_1': 'agent_2'}
    )

In [ ]:
def execute_graph(messages_list) -> str:

    builder = StateGraph(state_schema=CustomState)
    builder.add_node('agent_1', agent_1)
    builder.add_node('agent_2', agent_2)

    builder.add_edge(START, 'agent_1')

    graph = builder.compile()
    output = graph.invoke({'messages': messages_list})

    return output['messages'][-1].content
